# Setup

In [1]:
import os
import shutil
from datetime import datetime

## Configuration

In [2]:
WORKING_DIR = "/kaggle/working/"
CODE_DIR = "/kaggle/temp/src"
GDRIVE_DIR = "/content/drive/MyDrive/AIO25-MIX002/Projects/AIO2025_Conquer_CONQ008_M7_Project/"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download code

In [4]:
BRANCH = "ngocdung/make-inference-n-submit"

In [5]:
!ls /kaggle/working/data

ls: cannot access '/kaggle/working/data': No such file or directory


In [6]:
# If directory "src" not exist then clone a new one
!pwd
![ -d "{CODE_DIR}" ] || git clone --depth 1  --branch "{BRANCH}" "https://github.com/aio25-mix002/m07-p7.1" "{CODE_DIR}"


/content
Cloning into '/kaggle/temp/src'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 1), reused 10 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 21.62 KiB | 21.62 MiB/s, done.
Resolving deltas: 100% (1/1), done.


## Fetch the latest code

In [7]:
# Go to CODE_DIR, Fetch the latest code
%cd {CODE_DIR}
!git clean -fdx
!git status
!git pull
!pwd


/kaggle/temp/src
On branch ngocdung/make-inference-n-submit
Your branch is up to date with 'origin/ngocdung/make-inference-n-submit'.

nothing to commit, working tree clean
Already up to date.
/kaggle/temp/src


# Data Preparation

### Option 1: Using Kaggle API Credentials (Only if data in Google Drive not available)

First, ensure you have downloaded your `kaggle.json` API token from your Kaggle account. Once downloaded, upload it to your Colab session. You can do this via the 'Files' tab on the left sidebar.

After uploading, we will move it to the correct directory (`~/.kaggle/`) and set the necessary permissions.

In [8]:
import os

# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle

# Move the uploaded kaggle.json file to the .kaggle directory
# Assuming kaggle.json is in the current working directory after upload
# If you uploaded it to a different path, please adjust '/content/kaggle.json'
!mv /content/kaggle.json ~/.kaggle/kaggle.json

# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

print('Kaggle API credentials set up successfully!')
!ls -la ~/.kaggle/kaggle.json

Kaggle API credentials set up successfully!
-rw------- 1 root root 67 Jan  9 00:35 /root/.kaggle/kaggle.json


In [ ]:
# !python download_data.py
!kaggle competitions download -c action-video
!unzip action-video.zip -q -d {WORKING_DIR}

In [22]:
!ls {WORKING_DIR}data/test

0    127  156  185  213  242  271  30	329  358  387  415  444  473  501  72
1    128  157  186  214  243  272  300	33   359  388  416  445  474  502  73
10   129  158  187  215  244  273  301	330  36   389  417  446  475  503  74
100  13   159  188  216  245  274  302	331  360  39   418  447  476  504  75
101  130  16   189  217  246  275  303	332  361  390  419  448  477  505  76
102  131  160  19   218  247  276  304	333  362  391  42   449  478  506  77
103  132  161  190  219  248  277  305	334  363  392  420  45	 479  507  78
104  133  162  191  22	 249  278  306	335  364  393  421  450  48   508  79
105  134  163  192  220  25   279  307	336  365  394  422  451  480  509  8
106  135  164  193  221  250  28   308	337  366  395  423  452  481  51   80
107  136  165  194  222  251  280  309	338  367  396  424  453  482  52   81
108  137  166  195  223  252  281  31	339  368  397  425  454  483  53   82
109  138  167  196  224  253  282  310	34   369  398  426  455  484  54   83
11  

In [32]:
# !zip -r /kaggle/working/mini_data.zip /kaggle/working/mini_data
!ls -lh {WORKING_DIR}

total 127M
drwxr-xr-x 4 root root 4.0K Jan  9 00:39 data
drwxr-xr-x 4 root root 4.0K Jan  9 02:11 mini_data
-rw-r--r-- 1 root root 127M Jan  9 02:13 mini_data.zip
-rw-r--r-- 1 root root 4.4K Dec 12 10:45 submission_template.csv
drwxr-xr-x 2 root root 4.0K Jan  9 00:39 weights


In [26]:
!ls {WORKING_DIR}data/test/0

10000.jpg  10003.jpg  10006.jpg  10009.jpg  10012.jpg  10015.jpg
10001.jpg  10004.jpg  10007.jpg  10010.jpg  10013.jpg
10002.jpg  10005.jpg  10008.jpg  10011.jpg  10014.jpg


### (Unused due to synchronizing problem) Option 2: Using data saved in Google Drive


In [10]:
# import os
# import shutil

# source_gdrive_data_path = os.path.join(GDRIVE_DIR, 'Data')
# destination_working_data_path = os.path.join(WORKING_DIR, 'data')

# print(f"Attempting to copy data from Google Drive: {source_gdrive_data_path}")
# print(f"To working directory: {destination_working_data_path}")

# try:
#     # Create the destination directory if it doesn't exist. If it exists, remove it first to avoid errors.
#     if os.path.exists(destination_working_data_path):
#         print(f"Destination directory {destination_working_data_path} already exists. Removing before copy...")
#         shutil.rmtree(destination_working_data_path)

#     shutil.copytree(source_gdrive_data_path, destination_working_data_path)
#     print(f"Successfully copied '{source_gdrive_data_path}' to '{destination_working_data_path}'")
# except FileNotFoundError:
#     print(f"Error: Source data directory not found in Google Drive at {source_gdrive_data_path}")
# except Exception as e:
#     print(f"An error occurred while copying the data from Google Drive: {e}")

# Train

In [11]:
!python train.py

Using device: cuda
Initializing datasets...
model.safetensors: 100% 346M/346M [00:02<00:00, 126MB/s]
Loaded pretrained weights. Missing: 132, Unexpected: 0

Epoch 1/10
Train Loss: 3.6898 | Acc: 0.0915
Val Loss: 3.5272   | Acc: 0.1198
New best model saved! (0.1198)

Epoch 2/10
Train:  41% 579/1407 [05:10<07:22,  1.87it/s, acc=0.1291, loss=3.3721]Exception in thread Thread-5 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 52, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib

Save the best checkpoint to Google Drive

In [12]:
import os
import shutil
from datetime import datetime

# Define the brief note for the filename
NOTE = "vanilla" # !!! Should edit every new run
SAVE_PATH = f"{GDRIVE_DIR}Artifacts/Checkpoints"


In [13]:

# Define the source path of the best model checkpoint
source_checkpoint_path = os.path.join(CODE_DIR, 'checkpoints', 'best_model.pth')

# Define your Google Drive destination folder path
# IMPORTANT: Please replace 'YOUR_GOOGLE_DRIVE_FOLDER_PATH' with the actual path to your folder in Google Drive.
# For example, it might be '/content/drive/MyDrive/MyProjectCheckpoints/'

# Ensure the Google Drive folder exists
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
    print(f"Created Google Drive folder: {SAVE_PATH}")

# Generate a timestamp for the filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Construct the new filename for the checkpoint
new_checkpoint_filename = f"model_{timestamp}_{NOTE}.pth"
destination_checkpoint_path = os.path.join(SAVE_PATH, new_checkpoint_filename)

# Copy the checkpoint
try:
    shutil.copy(source_checkpoint_path, destination_checkpoint_path)
    print(f"Checkpoint successfully saved to: {destination_checkpoint_path}")
except FileNotFoundError:
    print(f"Error: Source checkpoint not found at {source_checkpoint_path}")
except Exception as e:
    print(f"An error occurred while saving the checkpoint: {e}")


Checkpoint successfully saved to: /content/drive/MyDrive/AIO25-MIX002/Projects/AIO2025_Conquer_CONQ008_M7_Project/Artifacts/Checkpoints/model_20260109_005840_vanilla.pth


Reload the checkpoint (if needed)

In [15]:
import torch
import os

# # Define the path to the checkpoint file
destination_checkpoint_path = source_checkpoint_path #f'{GDRIVE_DIR}Artifacts/Checkpoints/model_20260107_081244_vanilla.pth'

# Check if the checkpoint file exists
if os.path.exists(destination_checkpoint_path):
    # Load the checkpoint
    loaded_checkpoint = torch.load(destination_checkpoint_path, map_location=torch.device('cpu')) # Use 'cuda' if you want to load to GPU
    print(f"Checkpoint loaded successfully from: {destination_checkpoint_path}")
    print("Keys in the loaded checkpoint:", loaded_checkpoint.keys())

    # Example of how you might load it into a model (assuming 'model' is defined)
    # model.load_state_dict(loaded_checkpoint['model_state_dict'])
    # optimizer.load_state_dict(loaded_checkpoint['optimizer_state_dict'])
    # epoch = loaded_checkpoint['epoch']
    # loss = loaded_checkpoint['loss']
else:
    print(f"Error: Checkpoint not found at {destination_checkpoint_path}")


Checkpoint loaded successfully from: /kaggle/temp/src/checkpoints/best_model.pth
Keys in the loaded checkpoint: odict_keys(['smif.alpha', 'smif.conv_fuse.weight', 'smif.conv_fuse.bias', 'backbone.cls_token', 'backbone.pos_embed', 'backbone.patch_embed.proj.weight', 'backbone.patch_embed.proj.bias', 'backbone.blocks.0.norm1.weight', 'backbone.blocks.0.norm1.bias', 'backbone.blocks.0.attn.qkv.weight', 'backbone.blocks.0.attn.qkv.bias', 'backbone.blocks.0.attn.proj.weight', 'backbone.blocks.0.attn.proj.bias', 'backbone.blocks.0.norm2.weight', 'backbone.blocks.0.norm2.bias', 'backbone.blocks.0.mlp.fc1.weight', 'backbone.blocks.0.mlp.fc1.bias', 'backbone.blocks.0.mlp.fc2.weight', 'backbone.blocks.0.mlp.fc2.bias', 'backbone.blocks.0.lmim.delta', 'backbone.blocks.0.lmim.reduce.weight', 'backbone.blocks.0.lmim.reduce.bias', 'backbone.blocks.0.lmim.expand.weight', 'backbone.blocks.0.lmim.expand.bias', 'backbone.blocks.0.lmim.temporal_mlp.0.weight', 'backbone.blocks.0.lmim.temporal_mlp.0.bias', 

# Submission

Inference of Test set

In [21]:
!python inference.py --checkpoint {destination_checkpoint_path} --data_root {WORKING_DIR}data/test

Using device: cuda
INFERENCE ON TEST SET
Loading checkpoint from /kaggle/temp/src/checkpoints/best_model.pth...
Model loaded

Loading test dataset...
Test samples: 0


Make submission file and submit to Kaggle